Date Crawl Heatlhcare: 21/10/2017

<h3> Utils </h3>

In [1]:
execfile("utils.py")

In [2]:
sector = "healthcare"

<h3> News Data </h3>

In [8]:
def get_news(ticker, sector):
    response = intrinio_api_request(ticker)
    response_to_file(response, os.path.join("..", "data", sector, "news", ticker + ".csv"))
    
def all_news_sector(sector):
    tickers = load_tickers(sector)
    map(lambda x: get_news(x, sector), tickers)

In [ ]:
all_news_sector(sector)

<h3> Google Finance Historical Data </h3>

In [23]:
def get_stock_tick(ticker, sector):
    try:
        response = google_request(ticker)
        response_to_file(response, os.path.join("..", "data", sector, "stocks", ticker + ".csv"))
    except:
        print "Error"  

def all_stocks_sector(sector):
    tickers = load_tickers(sector)
    map(lambda x: get_stock_tick(x, sector), tickers)

In [24]:
all_stocks_sector(sector)

https://finance.google.com/finance/historical?q=AGN&startdate=Jan%201,%202014&output=csv
Error
https://finance.google.com/finance/historical?q=NVRO&startdate=Jan%201,%202014&output=csv
https://finance.google.com/finance/historical?q=NUVA&startdate=Jan%201,%202014&output=csv
https://finance.google.com/finance/historical?q=NBIX&startdate=Jan%201,%202014&output=csv
https://finance.google.com/finance/historical?q=MGLN&startdate=Jan%201,%202014&output=csv
https://finance.google.com/finance/historical?q=AAAP&startdate=Jan%201,%202014&output=csv
https://finance.google.com/finance/historical?q=XON&startdate=Jan%201,%202014&output=csv
https://finance.google.com/finance/historical?q=ALKS&startdate=Jan%201,%202014&output=csv
https://finance.google.com/finance/historical?q=DVA&startdate=Jan%201,%202014&output=csv
https://finance.google.com/finance/historical?q=AMGN&startdate=Jan%201,%202014&output=csv
https://finance.google.com/finance/historical?q=EVHC&startdate=Jan%201,%202014&output=csv
https:/

https://finance.google.com/finance/historical?q=CVS&startdate=Jan%201,%202014&output=csv
https://finance.google.com/finance/historical?q=BAX&startdate=Jan%201,%202014&output=csv
https://finance.google.com/finance/historical?q=RARE&startdate=Jan%201,%202014&output=csv
https://finance.google.com/finance/historical?q=REGN&startdate=Jan%201,%202014&output=csv
https://finance.google.com/finance/historical?q=BMRN&startdate=Jan%201,%202014&output=csv
https://finance.google.com/finance/historical?q=PDCO&startdate=Jan%201,%202014&output=csv
https://finance.google.com/finance/historical?q=WBA&startdate=Jan%201,%202014&output=csv
https://finance.google.com/finance/historical?q=ACAD&startdate=Jan%201,%202014&output=csv
https://finance.google.com/finance/historical?q=ICUI&startdate=Jan%201,%202014&output=csv
https://finance.google.com/finance/historical?q=MCK&startdate=Jan%201,%202014&output=csv
Error
https://finance.google.com/finance/historical?q=CRL&startdate=Jan%201,%202014&output=csv
https://f

<h3> News Model </h3>

In [3]:
import pandas as pd

def load_news(sector):
    tickers = load_tickers(sector)
    df_news = pd.DataFrame()
    for t in tickers:
        df_news = df_news.append(pd.read_csv(os.path.join("..", "data", sector, "news", t + ".csv"), header=1))
    return df_news

In [9]:
df_news = load_news(sector)
df_news = df_news.drop_duplicates(['TITLE'])

In [10]:
df_news.SUMMARY.head().values

array([ 'Allergan stock has fallen ~15.4% in 3Q17. It has fallen 9.8% year-to-date as of October 20, 2017.',
       'Another analyst is weighing in on the growing guesswork regarding the likelihood that Allergan (AGN) could be heading for a break up.  Allergan is appealing the decision, though the outcome of such a move is far from certain.  Meanwhile, several analysts warn that a cheaper-priced generic version of Restasis could hit the U.S. market next year, eroding sales of Allergan\xe2\x80\x99s second-biggest selling drug and creating a $1.5 billion revenue hole.',
       'Johnson amp Johnson JNJ kicked off third quarter earnings season for the pharmaceutical sector on a strong note with the company beating on both earnings and revenues While focus remained on J amp J s strong earnings results companies like Allergan AGN were also in the news regarding',
       "San Diego-based Imprimis' therapy will cost 99 cents for a month's supply, with refills starting at $79 a month and the co

In [105]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'[a-zA-Z]+')
df_news['SUMMARY_NEW'] = df_news.SUMMARY.map(lambda x: tokenizer.tokenize(str(x).lower()))
df_news.SUMMARY_NEW.head()

0     [allergan, stock, has, fallen, 15, 4, in, 3q17...
2     [another, analyst, is, weighing, in, on, the, ...
4     [johnson, amp, johnson, jnj, kicked, off, thir...
8     [san, diego, based, imprimis, therapy, will, c...
10    [top, health, care, stockstop, health, care, s...
Name: SUMMARY, dtype: object

In [106]:
from stop_words import get_stop_words
en_stop = [str(x) for x in get_stop_words('en')]

df_news['SUMMARY_STOPPED'] = df_news.SUMMARY_NEW.map(lambda x: [w for w in x if not w in en_stop])
df_news.SUMMARY_STOPPED.head()

0     [allergan, stock, fallen, 15, 4, 3q17, fallen,...
2     [another, analyst, weighing, growing, guesswor...
4     [johnson, amp, johnson, jnj, kicked, third, qu...
8     [san, diego, based, imprimis, therapy, will, c...
10    [top, health, care, stockstop, health, care, s...
Name: SUMMARY_STOPPED, dtype: object

In [107]:
from nltk.stem.porter import PorterStemmer
p_stemmer = PorterStemmer()

def stem_word(w):
    try:
        return p_stemmer.stem(w.decode('utf-8'))
    except:
        return None    

df_news['SUMMARY_STEMMED'] = df_news.SUMMARY_STOPPED.map(lambda x: [stem_word(w) for w in x])
df_news.SUMMARY_STEMMED.head()

0     [allergan, stock, fallen, 15, 4, 3q17, fallen,...
2     [anoth, analyst, weigh, grow, guesswork, regar...
4     [johnson, amp, johnson, jnj, kick, third, quar...
8     [san, diego, base, imprimi, therapi, will, cos...
10    [top, health, care, stockstop, health, care, s...
Name: SUMMARY_STEMMED, dtype: object

In [108]:
from gensim import corpora, models

df_news.SUMMARY_CLEAN = df_news.SUMMARY_STEMMED.map(lambda x: [w for w in x if w is not None])
dictionary = corpora.Dictionary(df_news.SUMMARY_CLEAN)
df_news.SUMMARY_CLEAN.head()

0     [allergan, stock, fallen, 15, 4, 3q17, fallen,...
2     [anoth, analyst, weigh, grow, guesswork, regar...
4     [johnson, amp, johnson, jnj, kick, third, quar...
8     [san, diego, base, imprimi, therapi, will, cos...
10    [top, health, care, stockstop, health, care, s...
Name: SUMMARY_STEMMED, dtype: object

In [109]:
corpus = [dictionary.doc2bow(x) for x in df_news.SUMMARY_CLEAN.values]
corpus[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 2),
 (11, 1),
 (12, 1)]

In [110]:
ldamodel = models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, passes=20)
print(ldamodel.print_topics(num_topics=3, num_words=5))

[(0, u'0.025*"s" + 0.025*"stock" + 0.016*"share" + 0.015*"trade" + 0.010*"market"'), (1, u'0.018*"announc" + 0.017*"s" + 0.014*"compani" + 0.012*"today" + 0.011*"inc"'), (2, u'0.028*"inc" + 0.021*"earn" + 0.017*"2016" + 0.017*"report" + 0.015*"2017"')]


In [113]:
ldamodel.print_topics(num_topics=10, num_words=30)

[(0,
  u'0.025*"s" + 0.025*"stock" + 0.016*"share" + 0.015*"trade" + 0.010*"market" + 0.008*"week" + 0.008*"one" + 0.007*"investor" + 0.006*"nasdaq" + 0.006*"price" + 0.006*"amp" + 0.006*"etf" + 0.005*"day" + 0.005*"new" + 0.005*"year" + 0.005*"p" + 0.005*"symbol" + 0.005*"today" + 0.005*"index" + 0.005*"buy" + 0.005*"look" + 0.005*"chang" + 0.004*"inc" + 0.004*"1" + 0.004*"160" + 0.004*"2" + 0.004*"gain" + 0.004*"compani" + 0.004*"among" + 0.004*"contract"'),
 (1,
  u'0.018*"announc" + 0.017*"s" + 0.014*"compani" + 0.012*"today" + 0.011*"inc" + 0.010*"health" + 0.009*"2017" + 0.009*"2016" + 0.009*"drug" + 0.008*"noodl" + 0.008*"new" + 0.008*"busi" + 0.007*"will" + 0.007*"prnewswir" + 0.005*"pharmaceut" + 0.005*"nyse" + 0.005*"global" + 0.005*"plan" + 0.005*"wire" + 0.005*"centen" + 0.004*"said" + 0.004*"newswir" + 0.004*"patient" + 0.004*"insur" + 0.004*"u" + 0.004*"develop" + 0.004*"corpor" + 0.004*"york" + 0.004*"nasdaq" + 0.004*"provid"'),
 (2,
  u'0.028*"inc" + 0.021*"earn" + 0.01

<h3> Sentiment Analysis </h3>

In [117]:
import pysentiment as ps
lm = ps.LM()

df_news.SUMMARY_SCORES = df_news.SUMMARY.map(lambda x: lm.get_score(lm.tokenize(str(x))))
df_news.SUMMARY_SCORES.head()

0     {u'Polarity': 0.0, u'Positive': 0, u'Negative'...
2     {u'Polarity': -0.999999666667, u'Positive': 0,...
4     {u'Polarity': 0.0, u'Positive': 0, u'Negative'...
8     {u'Polarity': -0.999999000001, u'Positive': 0,...
10    {u'Polarity': 0.999999000001, u'Positive': 1, ...
Name: SUMMARY, dtype: object

In [118]:
df_news.SUMMARY_SCORES.head().values

array([{'Polarity': 0.0, 'Positive': 0, 'Negative': 0, 'Subjectivity': 0.0},
       {'Polarity': -0.9999996666667778, 'Positive': 0, 'Negative': 3, 'Subjectivity': 0.09090908815427007},
       {'Polarity': 0.0, 'Positive': 0, 'Negative': 0, 'Subjectivity': 0.0},
       {'Polarity': -0.9999990000010001, 'Positive': 0, 'Negative': 1, 'Subjectivity': 0.032258063475546335},
       {'Polarity': 0.9999990000010001, 'Positive': 1, 'Negative': 0, 'Subjectivity': 0.031249999023437534}], dtype=object)